### DQN Docking Case

In [14]:
# setup
import gym
import gym_space_docking

print(gym_space_docking)
env = gym.make('space_docking-v0')
print(env)


ModuleNotFoundError: No module named 'gym_space_docking'

### create enviroment pygame

ModuleNotFoundError: No module named 'pygame'

In [ ]:
# create network

